# Getting Subreddit Submissions from Reddit 

We used PRAW to get submissions in order to get enough comments and questions for at least 1,000 submissions from each subreddit.

In [1]:
import pandas as pd
import numpy as np
import datetime
import praw
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
import re

import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

In [2]:
reddit = praw.Reddit(
    #personal use script
    client_id = 'sQ-FPslTJiefaA',
    #secret
    client_secret = 'uIxv_Poyngp2l7yUMZca4lMKx4esIA',
    #account password
    password = 'T4sK5Ge&QiXYt!6',
    username = 'Designer_Staff_1284',
    user_agent = 'attempt three by u/Designer_Staff_1284 ' #description by my name
)

In [3]:
previous = pd.read_csv('previous.csv')

In [4]:
previous.head()

,id,title,num_comments,body,num_upvotes,time_posted,time_now,time_delta,subreddit,top_comment,text_length
0,nb0592,how common is it to not have your country's mo...,3,"/r/titlegoretaken spain as a whole, the most w...",3,2021-05-12 21:37:16,2021-05-13 01:20:57.717158,0 days 03:43:41.717163,1,"[""About 40% of the population speaks French. I...",318
1,nb3wei,most popular fast food restaurant/take-away?,14,hey guys i’m not from usa but i have seen many...,3,2021-05-13 00:31:12,2021-05-13 01:20:57.864588,0 days 00:49:45.864594,0,"[""McDonalds, Subway and Starbucks are probably...",22
2,nb339e,is it true that you guys don't like/go to mcdo...,55,here in my country they're one of the most pop...,3,2021-05-12 23:52:33,2021-05-13 01:20:58.146242,0 days 01:28:25.146249,0,['It’s not my first choice by far. I’d rather ...,49
3,nb2x64,why were football fans so upset by the europea...,3,it seemed to me like it would increase the lev...,0,2021-05-12 23:44:35,2021-05-13 01:20:58.274452,0 days 01:36:23.274458,1,"['Cause fans don\'t want ""top-notch matches"" i...",25
4,nb0xx6,why do people stick four fingers out?,29,usually it's people in a lifted truck who do i...,1,2021-05-12 22:13:39,2021-05-13 01:20:58.525647,0 days 03:07:19.525654,0,['Are you talking about waving? Do we serious...,17


In [5]:
previous.shape

(1428, 11)

In [7]:
ids = []
titles = [] 
bodies = []
subreddits = []
num_comments = []
num_upvotes = []
time_posted = []
time_now = []
time_delta = []
top_comment = []
for submission in reddit.subreddit('AskAnAmerican').top('year', limit = 1_000):
    submission.comment_sort = 'best'
    submission.comment_limit = 2

    if submission.title not in previous.title.values: 
        
        ids.append(submission.id)
        titles.append(submission.title) 
        bodies.append(submission.selftext)
        subreddits.append(submission.subreddit)
        num_comments.append(submission.num_comments)
        num_upvotes.append(submission.ups)
        time_posted.append(submission.created_utc)
        time_now.append(datetime.datetime.utcnow())
        time_delta.append(datetime.datetime.utcnow() - \
                              datetime.datetime.utcfromtimestamp(submission.created_utc))
        top_comment.append('' if not submission.comments \
                     else [comment.body \
                           for comment in submission.comments \
                           if (hasattr(comment, 'body') \
                           and comment.stickied == False)][0])
america_df = pd.DataFrame({'id': ids, 
                       'title': titles,
                      'num_comments': num_comments,
                       'body': bodies,
                      'num_upvotes': num_upvotes,
                      'time_posted': time_posted,
                      'time_now': time_now,
                      'time_delta': time_delta,
                      'subreddit': subreddits,
                      'top_comment': top_comment})

In [8]:

# instantiate the lists
ids = []
titles = [] 
bodies = []
subreddits = []
num_comments = []
num_upvotes = []
time_posted = []
time_now = []
time_delta = []
top_comment = []
for submission in reddit.subreddit('AskEurope').top('year', limit = 1_000):
    submission.comment_sort = 'best'
    submission.comment_limit = 2

    if submission.title not in previous.title.values: 
        
        ids.append(submission.id)
        titles.append(submission.title) 
        bodies.append(submission.selftext)
        subreddits.append(submission.subreddit)
        num_comments.append(submission.num_comments)
        num_upvotes.append(submission.ups)
        time_posted.append(submission.created_utc)
        time_now.append(datetime.datetime.utcnow())
        time_delta.append(datetime.datetime.utcnow() - \
                              datetime.datetime.utcfromtimestamp(submission.created_utc))
        top_comment.append('' if not submission.comments \
                     else [comment.body \
                           for comment in submission.comments \
                           if (hasattr(comment, 'body') \
                           and comment.stickied == False)][0])
europe_df = pd.DataFrame({'id': ids, 
                       'title': titles,
                      'num_comments': num_comments,
                       'body': bodies,
                      'num_upvotes': num_upvotes,
                      'time_posted': time_posted,
                      'time_now': time_now,
                      'time_delta': time_delta,
                      'subreddit': subreddits,
                      'top_comment': top_comment})

In [9]:
merged_df = pd.concat([america_df, europe_df])

In [10]:
merged_df.shape

(1997, 10)

In [11]:
print(len(ids),
      len(titles),
      len(subreddits),
      len(num_comments),
      len(num_upvotes),
      len(time_posted),
      len(time_now),
      len(time_delta))
#      len(top_comment))

998 998 998 998 998 998 998 998


In [146]:
# america_df.shape, europe_df.shape

((996, 10), (995, 10))

In [115]:
# america_df['subreddit'].value_counts()

AskAnAmerican    51
AskEurope        49
Name: subreddit, dtype: int64

In [116]:
# df['subreddit'].iloc[6]

Subreddit(display_name='AskEurope')

In [117]:
# df['top_comment'].iloc[6]

'Same thing as always. By abusing of populist propaganda, and by taking advantage from the fear of the people by turning it into racism, sexism and homophobia ( seriously, about 60% of their propaganda is fake and people somehow still believes it [like seriously wtf how tf can someone unironically think we can go back to the 1600 empire days in 2021??? ]). Aaaaand of course, because once again, the left isnt doing a shit to stop it. They are actually not doing anything at all. At this point I wouldnt be surprised if, in a few years, we are being ruled by fascists. Once more, I mean.'

In [12]:
merged_df['text_length'] = [len(body.split(' ')) for body in merged_df['body']]

In [13]:
merged_df = merged_df[merged_df['text_length'] != 1]

In [15]:
merged_df.subreddit.value_counts()

AskAnAmerican    740
AskEurope        730
Name: subreddit, dtype: int64

In [16]:
merged_df['subreddit'].replace({'AskAnAmerican':0, 'AskEurope':1}, inplace = True)

In [17]:
merged_df['subreddit'].value_counts()

0    740
1    730
Name: subreddit, dtype: int64

In [124]:
# # drop duplicates
# print('Shape w/duplicates:', df.shape)
# df.drop_duplicates(subset = ['title', 'text_length', 'subreddit'], inplace = True)

# print('Shape w/o duplicates:', df.shape)

In [125]:
df['subreddit'].value_counts()

0    39
1    34
Name: subreddit, dtype: int64

In [126]:
df['subreddit'].isna().sum()

0

In [19]:
merged_df['time_posted'] = merged_df.time_posted.apply(lambda x: datetime.datetime.utcfromtimestamp(x))

In [20]:
merged_df.head()

,id,title,num_comments,body,num_upvotes,time_posted,time_now,time_delta,subreddit,top_comment,text_length
0,jpu7tu,Congratulations to President Elect Joe Biden a...,5060,Both Fox and CNN have called it for Biden. \n\...,29251,2020-11-07 16:52:48,2021-05-14 15:52:46.490986,187 days 22:59:58.490991,0,Georgia senate runoffs will be an absolute blo...,72
1,j5qqm1,Do you support the construction of a high-spee...,2753,[Here](https://i.insider.com/511262026bb3f78e2...,20282,2020-10-05 20:04:03,2021-05-14 15:52:46.710351,220 days 19:48:43.710355,0,I support high speed rail just make it regiona...,25
2,l530zd,Plans to put Harriet Tubman on the $20 bill ar...,1604,[https://finance.yahoo.com/news/harriet-tubman...,6909,2021-01-26 01:34:08,2021-05-14 15:52:46.967642,108 days 14:18:38.967647,0,I plan to continue using $20 bills.,3
3,m3gemo,Anyone else hasn’t got sick since the pandemic...,1161,I haven’t been sick with anything at all since...,6255,2021-03-12 12:19:32,2021-05-14 15:52:47.235304,63 days 03:33:15.235309,0,"Not sick once. But ""THE POLLENING"" (an ignor...",114
4,hbp7pd,July 4th - Is it OK for a foreigner to display...,822,I lived in the US for the last years and love ...,3145,2020-06-18 23:11:40,2021-05-14 15:52:47.433490,329 days 16:41:07.433494,0,If you're happy to be here and want to celebra...,105


In [129]:
for column in df.columns:
    print(df[column])

1     nbq272
2     nbppey
4     nbphjo
5     nbpgy6
8     nbjya4
       ...  
94    n9xifj
95    n9xw8i
96    n9vkj6
97    n9w0tz
98    n9wl1q
Name: id, Length: 73, dtype: object
1     Is the nine nations an accurate representation...
2             Is Texas a "Southern" or "Western" state?
4     What's the deal with the controversy about zon...
5     Is there often misunderstanding in spoken lang...
8     Does anybody use a reliable service that sends...
                            ...                        
94    Is getting charged per condiment packet common...
95              California’s how do you feel about Cox?
96    Selling your property in the North to buy a ni...
97    How much difference can it make the state you ...
98              Is it common to tell who you voted for?
Name: title, Length: 73, dtype: object
1      17
2      24
4       7
5      29
8       2
     ... 
94     35
95     56
96     86
97    563
98     95
Name: num_comments, Length: 73, dtype: int64
1     It wa

In [21]:
for column in ['title', 'body']:

    merged_df[column] = [re.sub('[\(\[].*?[\)\]]', '', t.replace('\n', '')) for t in merged_df[column]]
    merged_df[column] = [t.replace('/\\', ' ').replace(':-)', ' ').replace('tdb> ', ' ') 
                  for t in merged_df[column]]
    merged_df[column] = [t.replace('=', ' ').replace('--', ' ').replace('_', ' ')\
                  .replace('}', ' ').replace('*', ' ').replace('^', ' ')\
                  .replace('~', ' ')
                  for t in merged_df[column]]
    merged_df[column] = [t.replace('- < > -', ' ').replace('|', ' ').replace('*-', ' ') 
                  for t in merged_df[column]]

    merged_df[column] = [t.replace('...', ' ').replace('\t', ' ').lower().strip() 
                  for t in merged_df[column]]

In [22]:
previous.shape

(1428, 11)

# Combining DataFrames with Previous

In [23]:
combined = pd.concat([previous, merged_df])

In [24]:
combined.shape, merged_df.shape, previous.shape

((2898, 11), (1470, 11), (1428, 11))

In [25]:
combined.drop_duplicates(subset = ['title'],inplace = True)

In [26]:
combined.shape, merged_df.shape, previous.shape

((2683, 11), (1470, 11), (1428, 11))

# Saving to DF 

In [27]:
combined.to_csv('./previous.csv', index = False)